#chatbots



In [186]:
import numpy as np
import matplotlib.pyplot as plt
import spacy
from spacy import displacy
import nltk
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [187]:
import pickle
with open('/content/train_qa.txt','rb') as f:
  train_data = pickle.load(f)

In [188]:
with open('/content/test_qa.txt','rb') as f:
  test_data = pickle.load(f)

In [189]:
all_data  = train_data + test_data
# all_data

In [190]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [191]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [192]:
' '.join(train_data[0][2])

'n o'

In [193]:
df  = pd.DataFrame(all_data,columns=['Story','Question','Answer'])
df

,Story,Question,Answer
0,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Sandra, in, the, hallway, ?]",no
1,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bathroom, ?]",no
2,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, office, ?]",no
3,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bedroom, ?]",yes
4,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bedroom, ?]",yes
...,...,...,...
10995,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Mary, in, the, bedroom, ?]",no
10996,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Sandra, in, the, kitchen, ?]",no
10997,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Mary, in, the, bedroom, ?]",no
10998,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Sandra, in, the, garden, ?]",yes


In [194]:
vocab = set()
for story,question,answer in all_data:
  vocab = vocab.union(set(story))
  vocab = vocab.union(set(question))
vocab.add('yes')
vocab.add('no')

In [195]:
pd.DataFrame(vocab).T

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,Daniel,.,went,Mary,left,yes,no,travelled,down,garden,...,in,discarded,there,dropped,journeyed,hallway,office,to,bathroom,Sandra


In [196]:
vocab_len = len(vocab) + 1
# for i in range(len(all_data)):
#   for data in all_data[i]:
#     print(' '.join(data))

In [197]:
stories_len = [len(data[0]) for data in all_data]
question_len = [len(data[1]) for data in all_data]

In [198]:
maxlen_stories = max(stories_len)
maxlen_stories

156

In [199]:
maxlen_question = max(question_len)
maxlen_question

6

In [200]:
tokenizer = Tokenizer(filters='!"#$%&()*+-/:;<=>@[\\]^_`{|}~')

In [201]:
tokenizer.fit_on_texts(vocab)

In [202]:
tokenizer.word_index

{'daniel': 1,
 '.': 2,
 'went': 3,
 'mary': 4,
 'left': 5,
 'yes': 6,
 'no': 7,
 'travelled': 8,
 'down': 9,
 'garden': 10,
 'grabbed': 11,
 'kitchen': 12,
 'put': 13,
 'football': 14,
 'took': 15,
 'moved': 16,
 'milk': 17,
 'apple': 18,
 'picked': 19,
 'is': 20,
 'up': 21,
 'bedroom': 22,
 'the': 23,
 '?': 24,
 'back': 25,
 'got': 26,
 'john': 27,
 'in': 28,
 'discarded': 29,
 'there': 30,
 'dropped': 31,
 'journeyed': 32,
 'hallway': 33,
 'office': 34,
 'to': 35,
 'bathroom': 36,
 'sandra': 37}

In [203]:
train_story_text = []
train_question_text = []
train_answer_text = []
for s,q,a in train_data:
  train_story_text.append(s)
  train_question_text.append(q)
  train_answer_text.append(a)

In [204]:
# train_story_text

In [205]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)
# train_story_seq = np.array(train_story_seq)
# train_story_seq

In [206]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=maxlen_stories,max_question_len=maxlen_question):
    '''
    OUTPUT:
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []


    for story, query, answer in data:

        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        y = np.zeros(len(word_index) + 1)
        y[word_index[answer]] = 1

        X.append(x)
        Xq.append(xq)
        Y.append(y)

    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [207]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [208]:
import keras.layers
from keras.models import Sequential, Model
from keras.layers import Embedding,Input, Activation, Dense, Permute, Dropout,add, dot, concatenate,LSTM

In [209]:
input_sequence = Input((maxlen_stories,))
question_sequence = Input((maxlen_question,))
question_sequence

<KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'input_6')>

### input encoder M

In [210]:
# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len,output_dim=64))
input_encoder_m.add(Dropout(0.5))

# This encoder will output:
# (samples, story_maxlen, embedding_dim)

### input encoder C

In [211]:
# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len,output_dim=maxlen_question))
input_encoder_c.add(Dropout(0.5))
# output: (samples, story_maxlen, query_maxlen)

### question encoder


In [212]:
# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_len,
                               output_dim=64,
                               input_length=maxlen_question))
question_encoder.add(Dropout(0.5))
# output: (samples, query_maxlen, embedding_dim)

In [213]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question_sequence)

In [214]:
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

In [215]:
# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

In [216]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate_2')>

In [217]:
# Reduce with RNN (LSTM)
answer = LSTM(32)(answer)  # (samples, 32)
# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)  # (samples, vocab_size)
# probability distribution over the vocabulary
answer = Activation('softmax')(answer)

In [218]:
model = Model([input_sequence, question_sequence], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [219]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 156)]                0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 sequential_6 (Sequential)   (None, None, 64)             2432      ['input_5[0][0]']             
                                                                                                  
 sequential_8 (Sequential)   (None, 6, 64)                2432      ['input_6[0][0]']             
                                                                                            

In [220]:
model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=150,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/150
313/313 [==============================] - 5s 10ms/step - loss: 0.8336 - accuracy: 0.5002 - val_loss: 0.6943 - val_accuracy: 0.5030
Epoch 2/150
313/313 [==============================] - 2s 7ms/step - loss: 0.7011 - accuracy: 0.5080 - val_loss: 0.6949 - val_accuracy: 0.5030
Epoch 3/150
313/313 [==============================] - 2s 6ms/step - loss: 0.6966 - accuracy: 0.5053 - val_loss: 0.6932 - val_accuracy: 0.4970
Epoch 4/150
313/313 [==============================] - 2s 6ms/step - loss: 0.6961 - accuracy: 0.5043 - val_loss: 0.6950 - val_accuracy: 0.4970
Epoch 5/150
313/313 [==============================] - 2s 6ms/step - loss: 0.6965 - accuracy: 0.4961 - val_loss: 0.6947 - val_accuracy: 0.4970
Epoch 6/150
313/313 [==============================] - 2s 6ms/step - loss: 0.6950 - accuracy: 0.5091 - val_loss: 0.6947 - val_accuracy: 0.4970
Epoch 7/150
313/313 [==============================] - 2s 7ms/step - loss: 0.6963 - accuracy: 0.4929 - val_loss: 0.6933 - val_accuracy: 0.497

In [221]:
filename = 'chatbot.h5'
model.save(filename)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [222]:
model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

32/32 [==============================] - 0s 2ms/step


In [223]:
story =' '.join(word for word in test_data[1][0])
print(story)

Mary got the milk there . John moved to the bedroom . Mary discarded the milk . John went to the garden .


In [224]:
query = ' '.join(word for word in test_data[1][1])
print(query)

Is John in the kitchen ?


In [225]:
print("Answer is:",test_data[1][2])

Answer is: no


In [226]:
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  no
Probability of certainty was:  0.98773766


In [227]:
my_story = "John left the kitchen . Sandra dropped the football in the garden ."
my_story.split()

['John',
 'left',
 'the',
 'kitchen',
 '.',
 'Sandra',
 'dropped',
 'the',
 'football',
 'in',
 'the',
 'garden',
 '.']

In [228]:
my_question = "is sandra in the kitchen ?"
my_question.split()

['is', 'sandra', 'in', 'the', 'kitchen', '?']

In [229]:
mydata = [(my_story.split(),my_question.split(),'yes')]

In [230]:
my_story,my_ques,my_ans = vectorize_stories(mydata)

In [231]:
pred_results = model.predict(([ my_story, my_ques]))
pred_results

1/1 [==============================] - 0s 19ms/step


array([[6.4625823e-07, 6.0958536e-07, 6.0558892e-07, 5.4668311e-07,
        5.0562619e-07, 5.7854521e-07, 5.0225642e-02, 9.4975430e-01,
        6.6211840e-07, 4.6306127e-07, 5.4344520e-07, 6.7925527e-07,
        5.4317525e-07, 5.0117114e-07, 6.1868434e-07, 5.7458186e-07,
        5.7907124e-07, 4.2912677e-07, 4.7670451e-07, 6.0468057e-07,
        6.0906234e-07, 6.1060183e-07, 5.3535160e-07, 5.4721096e-07,
        5.7368061e-07, 4.2825349e-07, 5.1121572e-07, 5.7979901e-07,
        5.6752503e-07, 5.6164328e-07, 5.6442406e-07, 4.6170663e-07,
        4.9580632e-07, 8.0467947e-07, 5.6998425e-07, 4.4022880e-07,
        5.0185747e-07, 4.1981858e-07]], dtype=float32)

In [232]:
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  no
Probability of certainty was:  0.9497543
